# Create rotations chart data

# Setup

In [49]:
import pandas as pd
import numpy as np
import nba_api
import re

In [11]:
from nba_api.stats.static.teams import get_teams, find_teams_by_nickname
from nba_api.stats.endpoints import LeagueGameFinder
from nba_api.stats.endpoints import PlayByPlayV2
from nba_api.stats.endpoints import PlayByPlay
from nba_api.stats.endpoints import BoxScorePlayerTrackV2

In [12]:
spurs_info = find_teams_by_nickname('Spurs')
spurs_id = spurs_info[0]['id']

# find all Spurs games
gamefinder = LeagueGameFinder(team_id_nullable=spurs_id,
                                              season_nullable='2019-20')
team_box_scores_df = gamefinder.get_data_frames()[0]
team_box_scores_df.head(n=10)

spurs_games = list(team_box_scores_df.GAME_ID.unique()) # grab unique list of game IDs

test_game = spurs_games[0]

## Get list of active players and starters for each team

In [13]:
# get list of rosters and starting players
roster_data_df = BoxScorePlayerTrackV2(game_id=test_game).data_sets[0].get_data_frame()
starters_df = roster_data_df[roster_data_df.START_POSITION.isin(['F', 'G', 'C'])]
spurs_starters = list(starters_df[starters_df.TEAM_ABBREVIATION == 'SAS'].PLAYER_NAME)
opp_starters = list(starters_df[starters_df.TEAM_ABBREVIATION != 'SAS'].PLAYER_NAME)
spurs_roster = list(roster_data_df[roster_data_df.TEAM_ABBREVIATION == 'SAS'].PLAYER_NAME)
opp_roster = list(roster_data_df[roster_data_df.TEAM_ABBREVIATION != 'SAS'].PLAYER_NAME)

In [14]:
pbp = PlayByPlayV2(game_id=test_game).data_sets

In [15]:
pbp_df = pbp[0].get_data_frame()
pbp_df.head(n=10)

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,...,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,VIDEO_AVAILABLE_FLAG
0,0021901314,2,12,0,1,6:38 PM,12:00,None,None,None,...,None,None,0,0,None,NaN,None,None,None,0
1,0021901314,4,10,0,1,6:38 PM,12:00,Jump Ball Bradley vs. Poeltl: Tip to Ingles,None,None,...,Spurs,SAS,4,204060,Joe Ingles,1.610613e+09,Utah,Jazz,UTA,1
2,0021901314,7,1,47,1,6:38 PM,11:46,Clarkson 7' Turnaround Jump Shot (2 PTS) (Mitc...,None,None,...,Jazz,UTA,0,0,None,NaN,None,None,None,1
3,0021901314,9,2,78,1,6:38 PM,11:31,O'Neale BLOCK (1 BLK),None,MISS Samanic 4' Floating Jump Shot,...,None,None,4,1626220,Royce O'Neale,1.610613e+09,Utah,Jazz,UTA,1
4,0021901314,11,4,0,1,6:38 PM,11:28,Bradley REBOUND (Off:0 Def:1),None,None,...,None,None,0,0,None,NaN,None,None,None,1
5,0021901314,12,2,1,1,6:38 PM,11:23,MISS O'Neale 25' 3PT Jump Shot,None,None,...,None,None,0,0,None,NaN,None,None,None,1
6,0021901314,13,4,0,1,6:38 PM,11:17,None,None,Samanic REBOUND (Off:0 Def:1),...,None,None,0,0,None,NaN,None,None,None,1
7,0021901314,14,2,78,1,6:38 PM,11:12,None,None,MISS Walker IV 14' Floating Jump Shot,...,None,None,0,0,None,NaN,None,None,None,1
8,0021901314,15,4,0,1,6:38 PM,11:10,Bradley REBOUND (Off:0 Def:2),None,None,...,None,None,0,0,None,NaN,None,None,None,1
9,0021901314,16,2,1,1,6:39 PM,10:58,MISS Clarkson 25' 3PT Jump Shot,None,None,...,None,None,0,0,None,NaN,None,None,None,1


# Functions

In [16]:
def conv_time_to_sec(time):
    split_time = time.split(':')
    # convert time to seconds
    # seconds in elapsed time
    new_time = int(split_time[0]) * 60 + int(split_time[1])
    new_time = 12*60 - new_time
    return new_time

In [17]:
def parse_player_in(pbp_text):
    split_text = pbp_text.split('FOR')
    
    return (split_text[0][4:].strip())

In [18]:
def parse_player_out(pbp_text):
    split_text = pbp_text.split('FOR')
    
    return (split_text[1].strip())

# Parse PBP data

In [33]:
pbp_df[(pbp_df.PERIOD == 2) & (pbp_df.VISITORDESCRIPTION.str.contains('SUB'))]

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,...,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,VIDEO_AVAILABLE_FLAG
143,0021901314,205,8,0,2,7:13 PM,9:15,None,None,SUB: Johnson FOR Murray,...,Spurs,SAS,0,0,None,NaN,None,None,None,0
144,0021901314,206,8,0,2,7:13 PM,9:15,None,None,SUB: Walker IV FOR Belinelli,...,Spurs,SAS,0,0,None,NaN,None,None,None,0
177,0021901314,253,8,0,2,7:21 PM,6:04,None,None,SUB: Poeltl FOR Johnson,...,Spurs,SAS,0,0,None,NaN,None,None,None,0
178,0021901314,254,8,0,2,7:21 PM,6:04,None,None,SUB: Belinelli FOR Eubanks,...,Spurs,SAS,0,0,None,NaN,None,None,None,0
192,0021901314,279,8,0,2,7:25 PM,4:29,None,None,SUB: Murray FOR Samanic,...,Spurs,SAS,0,0,None,NaN,None,None,None,0
200,0021901314,293,8,0,2,7:26 PM,3:51,None,None,SUB: Eubanks FOR Metu,...,Spurs,SAS,0,0,None,NaN,None,None,None,0
210,0021901314,312,8,0,2,7:30 PM,2:46,None,None,SUB: Samanic FOR Walker IV,...,Spurs,SAS,0,0,None,NaN,None,None,None,0


In [34]:
spurs_roster

['Keldon Johnson',
 'Luka Samanic',
 'Jakob Poeltl',
 'Lonnie Walker IV',
 'Dejounte Murray',
 'Marco Belinelli',
 'Quinndary Weatherspoon',
 'Drew Eubanks',
 'Chimezie Metu',
 'DeMar DeRozan',
 'Rudy Gay',
 'Patty Mills',
 'Derrick White']

In [142]:
spurs_last_names = [' '.join(l_name.split(' ')[1:]) for l_name in spurs_roster]
spurs_re = "(" +  ')|('.join(spurs_last_names) + ")"
opp_last_names = [' '.join(l_name.split(' ')[1:]) for l_name in opp_roster]
opp_re = '(' + ')|('.join(opp_last_names) + ')'

In [143]:
spurs_re

'(Johnson)|(Samanic)|(Poeltl)|(Walker IV)|(Murray)|(Belinelli)|(Weatherspoon)|(Eubanks)|(Metu)|(DeRozan)|(Gay)|(Mills)|(White)'

In [111]:
opp_re

"(Ingles)|(O'Neale)|(Bradley)|(Clarkson)|(Mitchell)|(Niang)|(Oni)|(Davis)|(Morgan)|(Brantley)|(Tucker)|(Williams-Goss)|(Wright-Foreman)"

In [67]:
spurs_re

'(Johnson)|(Samanic)|(Poeltl)|(IV)|(Murray)|(Belinelli)|(Weatherspoon)|(Eubanks)|(Metu)|(DeRozan)|(Gay)|(Mills)|(White)'

In [107]:
opp_re

"Ingles|O'Neale|Bradley|Clarkson|Mitchell|Niang|Oni|Davis|Morgan|Brantley|Tucker|Williams-Goss|Wright-Foreman"

In [151]:
p = re.compile(spurs_re)
play = sec_q_pbp.VISITORDESCRIPTION.iloc[44]

In [152]:
play

"MISS Samanic 3' Driving Layup"

In [155]:
p.search(play).group()

'Samanic'

In [188]:
sec_q_pbp.HOMEDESCRIPTION.iloc[0]
s = re.compile(opp_re)

In [198]:
sec_q_starters = np.array()
np.insert(sec_q_starters, 'Davis')

TypeError: array() missing required argument 'object' (pos 1)

In [ ]:
def find_starters(period, storage_list, df, sas_or_opp):
    # check home or away
    # check which team is which
    if home_or_away.lower() == 'home':
        side == 'HOMEDESCRPTION'
    else:
        side == 'VISITORDESCRIPTION'
    period_pbp = df[(df.PERIOD == period) & (pbp_df[side].notnull())]
    
    match = 
    
    pass

In [205]:
# get 2nd quarter starters list

sec_q_starters = list()

sec_q_pbp = pbp_df[(pbp_df.PERIOD == 2) & (pbp_df.HOMEDESCRIPTION.notnull())].copy()

for play in range(len(sec_q_pbp)):
    play_descrip = sec_q_pbp.HOMEDESCRIPTION.iloc[play]
        
    match = s.search(play_descrip)
    try:
        player = match.group()
        print(player)
        if player in sec_q_starters:
            continue
        else:
            sec_q_starters.append(player)
    except:
        continue

Davis
Davis
Clarkson
Davis
Clarkson
Ingles
Davis
Ingles
Davis
Davis
Mitchell
O'Neale
Oni
Clarkson
Mitchell
Davis
Davis
Davis
Brantley
Bradley
O'Neale
Bradley
Bradley
O'Neale
Mitchell
Brantley
Bradley
Bradley
Brantley
Brantley
Mitchell
Niang
Clarkson
Clarkson
Oni
Tucker
Oni
Niang
Oni
Brantley
Bradley
Tucker
Brantley
Niang
Brantley
Brantley
Bradley
Tucker
Tucker
Niang
Niang
Tucker
Tucker
Brantley
Brantley
Bradley
Bradley


In [208]:
np.array(sec_q_starters)[:5]

array(['Davis', 'Clarkson', 'Ingles', 'Mitchell', "O'Neale"], dtype='<U8')

In [187]:
print(s.search(sec_q_pbp.HOMEDESCRIPTION.iloc[0]))

None


In [ ]:
prin

# Create rotations data for visualization

In [47]:
any(l_name in sec_q_pbp.iloc[0].VISITORDESCRIPTION for l_name in spurs_last_names)

True

In [31]:
# find starters for each team
# at the start of each quarter, track down the five players initally in the game